In [1]:
import pandas as pd
import numpy as np
import os
from xml.etree import cElementTree as ET
from dbfread import DBF
import time
import sqlite3

In [2]:
os.getcwd()

'/Users/ninarethaller'

In [3]:
os.chdir("/Users/ninarethaller/Downloads/")

Les année d'élection :
1965	1969	1974	1981	1988	1995	2002	2007	2012	2017

Pour 2017, on utilise la date la plus proche

### Creation de la table de taux de chomage

In [4]:
file = "TXCHO-DEP.xml"
tree_emploi = ET.parse(file)
root_emploi = tree_emploi.getroot()

In [5]:
columns_name = ['time_period', 'dept_lib','dept_nb', 'rate']
data_emploi = pd.DataFrame(columns=columns_name)

In [6]:
for dept in root_emploi:

    dept_lib = dept.attrib["TITLE"].split(" - ")[1].replace(" ","")
    dept_nb =  dept.attrib["DEPARTEMENT"]
    
    for serie in dept:
        value = serie.attrib["OBS_VALUE"]
        date  = serie.attrib["TIME_PERIOD"]
        
        it_dict = {'time_period':date, 'dept_lib':dept_lib, "dept_nb":dept_nb, 'rate':value}   
        data_emploi = data_emploi.append(it_dict, ignore_index=True)

In [7]:
np.unique(data_emploi["dept_nb"])

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95'], dtype=object)

In [8]:
data_emploi.head()

,time_period,dept_lib,dept_nb,rate
0,2016-Q3,Aveyron,12,7.3
1,2016-Q2,Aveyron,12,7.2
2,2016-Q1,Aveyron,12,7.4
3,2015-Q4,Aveyron,12,7.4
4,2015-Q3,Aveyron,12,7.6


Conserver seulements les années d'élections, et faire un taux de variation ante/post election
pour mesurer l'impact du quinquénat sur l'élection
On fait aussi un taux glissant de un an sur le chomage un an avant élection pour mesurer un impact local sur le chomage sur la dernière année

In [9]:
def retry_quarter(q, value):
    q1 = np.nan
    q2 = np.nan
    q3 = np.nan
    q4 = np.nan
    if q == "Q1":
        q1 = float(value)
    elif q == "Q2":
        q2 = float(value)
    elif q == "Q3":
        q3 = float(value)
    else :
        q4 = float(value)
    return q1, q2, q3, q4

In [10]:
# Reorganisation de la table en ligne par année
columns_name = ['time_period', 'dept_lib','dept_nb', 'Q1_rate','Q2_rate','Q3_rate','Q4_rate']
data_emploi_2 = pd.DataFrame(columns=columns_name)

for i in range(data_emploi.shape[0]):

    time_an = data_emploi.iloc[i, 0].split("-")[0]
    time_q = data_emploi.iloc[i, 0].split("-")[1]
    dept_lib = data_emploi.iloc[i, 1]
    dept_nb = data_emploi.iloc[i, 2]
    rate = data_emploi.iloc[i, 3]

    list_index = data_emploi_2[(data_emploi_2["time_period"]==time_an) & (data_emploi_2["dept_nb"]==dept_nb)].index
    
    q1, q2, q3, q4 = retry_quarter(time_q, rate) 
    if len(list_index) == 0 :
        
        data_emploi_2 = data_emploi_2.append({"time_period":time_an,"dept_lib":dept_lib,
                                             "dept_nb":dept_nb, "Q1_rate":q1, "Q2_rate":q2,
                                              "Q3_rate":q3, "Q4_rate":q4}, ignore_index=True)
    elif len(list_index) == 1: 
        
        if not np.isnan(q1):
            data_emploi_2.iloc[list_index.values, 3] = rate
        if not np.isnan(q2):
            data_emploi_2.iloc[list_index.values, 4] = rate
        if not np.isnan(q3):
            data_emploi_2.iloc[list_index.values, 5] = rate
        if not np.isnan(q4):  
            data_emploi_2.iloc[list_index.values, 6] = rate
        
    else :
        print("Erreur longueur de liste")
        print("Longueur de la liste %s" % len(list_index))
        print("Index du dataframe %s" %i)

In [11]:
data_emploi_2.head()

,time_period,dept_lib,dept_nb,Q1_rate,Q2_rate,Q3_rate,Q4_rate
0,2016,Aveyron,12,7.4,7.2,7.3,NaN
1,2015,Aveyron,12,7.3,7.4,7.6,7.4
2,2014,Aveyron,12,7.4,7.3,7.3,7.3
3,2013,Aveyron,12,7.5,7.6,7.5,7.3
4,2012,Aveyron,12,6.5,6.7,6.8,7.2


In [15]:
len(np.unique(data_emploi_2["dept_lib"]))

96

In [16]:
len(np.unique(data_emploi_2["time_period"]))

35

In [20]:
np.unique(data_emploi_2["time_period"])

array(['1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016'], dtype=object)

In [17]:
96*35

3360

In [18]:
data_emploi_2.shape

(3360, 7)

In [12]:
data_emploi_2.to_csv("data_emploi.csv")

Pour les variations ne devront t-on pas utiliser plutot moyenne annuelle??  
Pb pour les variations on commence en 82, on va encore perdre une année d'historique  
Essayer avec des chiffres national?  

In [15]:
# Creation des varitions de taux et conservation des années electorales
# 1981	1988	1995	2002	2007	2012	2017

### Etat Civil

In [21]:
file2 = "ETAT-CIVIL-DEP.xml"
tree_civil = ET.parse(file2)
root_civil = tree_civil.getroot()

In [22]:
def each_serie(root, list_serie) :
    
    iteration = 0
    for name_serie in list_serie:
        columns_name = ['time_period', 'dept_lib','dept_nb'] + [name_serie]
        data_serie = pd.DataFrame(columns=columns_name)
    
        for dept in root:

            dept_lib = dept.attrib["TITLE"].split(" - ")[1].strip(" ")
            dept_nb =  dept.attrib["DEPARTEMENT"]
            serie_lib = dept.attrib["TITLE"].split(" - ")[0].strip(" ")

            if serie_lib == name_serie :
                for serie in dept:
                    
                    serie_value = serie.attrib["OBS_VALUE"]
                    serie_date  = serie.attrib["TIME_PERIOD"]
                    it_dict = {'time_period': serie_date, 'dept_lib':dept_lib, "dept_nb":dept_nb, str(name_serie): serie_value} 

                    data_serie = data_serie.append(it_dict, ignore_index=True)
         
        iteration +=1
        data_serie.sort_values(by=["dept_nb","time_period"])
     
        if iteration==1:
            data = data_serie
        else :
            data = pd.merge(data, data_serie, how='left', 
                    on=['time_period', 'dept_lib','dept_nb'])
    
    print(data.shape)
    print(data.head())
    
    return data.sort_values(by=["dept_nb","time_period"])

In [23]:
serie_a_garder = ["Naissances domiciliées par département", 
                    "Nombre total de mariages domiciliés", "Décès domiciliés par département"]

data_etat_civil = each_serie(root_civil, serie_a_garder)

(4022, 6)
  time_period dept_lib dept_nb Naissances domiciliées par département  \
0        2015    Paris      75                                  28267   
1        2014    Paris      75                                  29134   
2        2013    Paris      75                                  28945   
3        2012    Paris      75                                  29291   
4        2011    Paris      75                                  30094   

  Nombre total de mariages domiciliés Décès domiciliés par département  
0                                 NaN                            13997  
1                               12201                            13487  
2                               11864                            13939  
3                               11445                            14114  
4                               11183                            13771  


In [27]:
np.unique(data_etat_civil["dept_nb"])

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95', '971', '972', '973',
       '974', '976'], dtype=object)

In [29]:
len(np.unique(data_etat_civil["time_period"]))

41

In [30]:
41*101

4141

In [25]:
data_etat_civil.head()

,time_period,dept_lib,dept_nb,Naissances domiciliées par département,Nombre total de mariages domiciliés,Décès domiciliés par département
2951,1975,Ain,01,5057,2512,3854
2950,1976,Ain,01,5095,2358,3687
2949,1977,Ain,01,5293,2426,3810
2948,1978,Ain,01,5243,2270,3825
2947,1979,Ain,01,5468,2226,3846


In [35]:
data_etat_civil[data_etat_civil["dept_nb"]=="971"]

,time_period,dept_lib,dept_nb,Naissances domiciliées par département,Nombre total de mariages domiciliés,Décès domiciliés par département
3953,1994,Guadeloupe,971,6458,1637,2203
3952,1995,Guadeloupe,971,6403,1569,2339
3951,1996,Guadeloupe,971,6461,1625,2335
3950,1997,Guadeloupe,971,6641,1674,2278
3949,1998,Guadeloupe,971,5707,1791,2376
3948,1999,Guadeloupe,971,6481,1704,2504
3947,2000,Guadeloupe,971,6664,1703,2505
3946,2001,Guadeloupe,971,6706,1696,2461
3945,2002,Guadeloupe,971,6110,1620,2639
3944,2003,Guadeloupe,971,6102,1517,2522


In [31]:
data_etat_civil.shape

(4022, 6)

### Population

In [65]:
def load_pop(start_year, end_year):
    iteration = 0
    for year in range(start_year, end_year, 1):
        
        if year in [2015, 2016, 2014, 1990]:
            data_pop = pd.read_excel("estim-pop-dep-sexe-gca-1975-2016.xls", sheetname=str(year), 
                             skiprows=4,  skip_footer=10, parse_cols=7,
                                names=["dept_nb", "dept_lib","0-19ans","20-39ans","40-59ans",
                                       "60-74ans","75+ans","Total"])
        
        elif year > 1989 : 
            data_pop = pd.read_excel("estim-pop-dep-sexe-gca-1975-2016.xls", sheetname=str(year), 
                             skiprows=4,  skip_footer=9, parse_cols=7,
                                names=["dept_nb", "dept_lib","0-19ans","20-39ans","40-59ans",
                                       "60-74ans","75+ans","Total"])
        else : 
            data_pop = pd.read_excel("estim-pop-dep-sexe-gca-1975-2016.xls", sheetname=str(year), 
                             skiprows=4,  skip_footer=4, parse_cols=7,
                                names=["dept_nb", "dept_lib","0-19ans","20-39ans","40-59ans",
                                       "60-74ans","75+ans","Total"])
        iteration += 1
        data_pop["time_period"]=str(year)
        print(data_pop.iloc[-1,:])
        
        if iteration == 1:
            data = data_pop
        else :
            data = pd.concat([data, data_pop])
    
    print(data.head())
    print(data.shape)
    
    return data.sort_values(by=["dept_nb","time_period"])

In [66]:
data_pop = load_pop(1975, 2017)

dept_nb                95
dept_lib       Val-d'Oise
0-19ans            293014
20-39ans           262872
40-59ans           181391
60-74ans            72473
75+ans              28545
Total              838295
time_period          1975
Name: 95, dtype: object
dept_nb                95
dept_lib       Val-d'Oise
0-19ans            294883
20-39ans           267373
40-59ans           186497
60-74ans            70684
75+ans              29597
Total              849034
time_period          1976
Name: 95, dtype: object
dept_nb                95
dept_lib       Val-d'Oise
0-19ans            296326
20-39ans           272059
40-59ans           192269
60-74ans            68280
75+ans              30757
Total              859691
time_period          1977
Name: 95, dtype: object
dept_nb                95
dept_lib       Val-d'Oise
0-19ans            297971
20-39ans           277552
40-59ans           197784
60-74ans            65995
75+ans              32223
Total              871525
time_period       

In [56]:
data_pop.iloc[-1,:]

dept_nb                95
dept_lib       Val-d'Oise
0-19ans            349825
20-39ans           328381
40-59ans           322013
60-74ans           150681
75+ans              71463
Total             1222363
time_period          2016
Name: 95, dtype: object

In [70]:
42*96

4032

### Etat Civil + population

In [69]:
print(data_pop.shape)
print(min(data_pop["time_period"]))
print(max(data_pop["time_period"]))
print(len(np.unique(data_pop["dept_nb"])))
print(len(np.unique(data_pop["time_period"])))

(4032, 9)
1975
2016
96
42


In [73]:
41*96

3936

In [71]:
print(data_etat_civil.shape)
print(min(data_etat_civil["time_period"]))
print(max(data_etat_civil["time_period"]))
print(len(np.unique(data_etat_civil["dept_nb"])))
print(len(np.unique(data_etat_civil["time_period"])))

(4022, 6)
1975
2015
101
41


In [74]:
data_civil_pop = pd.merge(data_pop, data_etat_civil, how='left', 
                    on=['time_period', 'dept_lib','dept_nb'])

In [75]:
data_civil_pop.head()

,dept_nb,dept_lib,0-19ans,20-39ans,40-59ans,60-74ans,75+ans,Total,time_period,Naissances domiciliées par département,Nombre total de mariages domiciliés,Décès domiciliés par département
0,01,Ain,121326,105393,80174,50440,18944,376277,1975,5057,2512,3854
1,01,Ain,122612,107766,82110,49288,19550,381326,1976,5095,2358,3687
2,01,Ain,123830,110580,84889,47456,20452,387207,1977,5293,2426,3810
3,01,Ain,124878,113543,87314,45970,21275,392980,1978,5243,2270,3825
4,01,Ain,125921,116298,89428,44775,21921,398343,1979,5468,2226,3846


In [77]:
print(data_civil_pop.shape[0])
print(len(np.unique(data_civil_pop["dept_nb"])))

4032
96


In [78]:
print(data_emploi_2.shape[0])
print(len(np.unique(data_emploi_2["dept_nb"])))

3360
96


In [79]:
data_civil_pop.to_csv("data_civil_pop.csv")

### Données harmonisée  1968-2013

In [224]:
table1 = DBF('rp19682013_individus_dbase/rp19682013.dbf')
table2 = DBF('rp19682013_individus_dbase/rp19682013_varlist.dbf',encoding='latin-1')
table3 = DBF('rp19682013_individus_dbase/rp19682013_varmod.dbf',encoding='latin-1')

In [220]:
len(table)

38676000

In [225]:
for re in table2 :
    print(re)
    print("\n")

OrderedDict([('VARIABLE', 'AGE_REV'), ('LIBELLE', 'Age r\x82volu'), ('TYPE', 'Caract\x8are'), ('LONGUEUR', '3')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('LIBELLE', 'Ann\x82e du recensement de population'), ('TYPE', 'Caract\x8are'), ('LONGUEUR', '4')])


OrderedDict([('VARIABLE', 'CSP'), ('LIBELLE', 'Cat\x82gorie socioprofessionnelle (en 8 postes)'), ('TYPE', 'Caract\x8are'), ('LONGUEUR', '2')])


OrderedDict([('VARIABLE', 'DEP_NAIS'), ('LIBELLE', 'D\x82partement de naissance'), ('TYPE', 'Caract\x8are'), ('LONGUEUR', '3')])


OrderedDict([('VARIABLE', 'DEP_RAN_15'), ('LIBELLE', 'D\x82partement de r\x82sidence ant\x82rieure (en g\x82ographie 2015)'), ('TYPE', 'Caract\x8are'), ('LONGUEUR', '3')])


OrderedDict([('VARIABLE', 'DEP_RES_15'), ('LIBELLE', 'D\x82partement de r\x82sidence (en g\x82ographie 2015)'), ('TYPE', 'Caract\x8are'), ('LONGUEUR', '3')])


OrderedDict([('VARIABLE', 'DEP_TRA_15'), ('LIBELLE', 'D\x82partement de travail (en g\x82ographie 2015)'), ('TYPE', 'Caract\x8are')

In [226]:
for re in table3 :
    print(re)
    print("\n")
    

OrderedDict([('VARIABLE', 'AGE_REV'), ('MODALITE', '000 \x85 120'), ('MODLIBELLE', 'de 0 \x85 120 ans')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '1968'), ('MODLIBELLE', 'Recensement de 1968')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '1975'), ('MODLIBELLE', 'Recensement de 1975')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '1982'), ('MODLIBELLE', 'Recensement de 1982')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '1990'), ('MODLIBELLE', 'Recensement de 1990')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '1999'), ('MODLIBELLE', 'Recensement de 1999')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '2008'), ('MODLIBELLE', 'Recensement de 2008')])


OrderedDict([('VARIABLE', 'AN_RECENS'), ('MODALITE', '2013'), ('MODLIBELLE', 'Recensement de 2013')])


OrderedDict([('VARIABLE', 'CSP'), ('MODALITE', '1'), ('MODLIBELLE', 'Agriculteurs')])


OrderedDict([('VARIABLE', 'CSP'), ('MODALITE', '2'), ('MODLIBELLE', "Artisan

In [244]:
conn = sqlite3.connect('rp19682013_individus_dbase/example.sqlite')
c = conn.cursor()

In [272]:
for row in c.execute('SELECT AN_RECENS,COUNT(AN_RECENS) FROM rp19682013 GROUP BY AN_RECENS'):
        print (row)

('1968', 3725151)
('1975', 3660591)
('1982', 4704215)
('1990', 5449400)
('1999', 6660423)
('2008', 7412933)
('2013', 7063287)


In [273]:
for row in c.execute('SELECT AN_RECENS,NES4,COUNT(NES4) FROM rp19682013 GROUP BY NES4, AN_RECENS'):
        print (row)

('1968', '1', 171510)
('1975', '1', 120761)
('1982', '1', 128678)
('1990', '1', 125746)
('1999', '1', 139231)
('2008', '1', 124055)
('2013', '1', 118417)
('1968', '2', 631622)
('1975', '2', 599112)
('1982', '2', 668216)
('1990', '2', 687533)
('1999', '2', 677028)
('2008', '2', 687500)
('2013', '2', 621147)
('1968', '3', 247232)
('1975', '3', 205829)
('1982', '3', 233376)
('1990', '3', 246949)
('1999', '3', 234302)
('2008', '3', 345660)
('2013', '3', 344872)
('1968', '4', 1181577)
('1975', '4', 1223288)
('1982', '4', 1650510)
('1990', '4', 2023168)
('1999', '4', 2599480)
('2008', '4', 3263416)
('2013', '4', 3101949)
('1968', '9', 1493210)
('1975', '9', 1511601)
('1982', '9', 2023435)
('1990', '9', 2366004)
('1999', '9', 3010382)
('2008', '9', 2992302)
('2013', '9', 2876902)


In [285]:
for row in c.execute('SELECT TYP_ACT,SUM(POND) FROM rp19682013 WHERE AN_RECENS="2013" GROUP BY TYP_ACT, AN_RECENS'):
        print (row)

('1', 26733241.530168764)
('2', 4175157.586438414)
('3', 5062741.779365853)
('5', 14240929.046412589)
('6', 15352801.949536229)


In [293]:
26733241.530168764+4175157.586438414+14240929.046412589

45149328.16301977

In [294]:
(4175157.586438414/45149328.16301977)*100

9.24744122739381

In [ ]:
for i in range(dataframe.shape):
    
    if data.loc(i,annee)==2016:
        data.loc(i,new_variable) = Nan
    else :
        data.loc(i,new_variable) =  data.loc(i,Q4) - data.loc(i-1,Q4)